# Best Practices for Model Evaluation and Hyperparameter Tuning

## Streamlining workflows with pipelines

When working on a dataset we saw that we have to reuse the parameters that were obtained during the fitting of the training data. We will learn how to use the **Pipeline** class in scikit-learn for this.

### Loading a dataset

We will be working with the **Breast Cancer Wisconsin** dataset which contains 569 samples of malignant and benign tumor cells.

We will read the dataset and split it into training and test sets

In [1]:
import pandas as pd

df = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', 
    header=None)

Now we assign the 30 features to a NumPy array x, using **LabelEncoder** we transform the class labels from their original string representation (**M** and **B**) to integers.

In [2]:
from sklearn.preprocessing import LabelEncoder

x = df.loc[:, 2:].values
y = df.loc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)

Now the two classes are encoded as 1 = M and 0 = B. To check if the behaviour is correct we can call the method on two dummy examples

In [3]:
le.transform(['M', 'B'])

array([1, 0])

Before constructing the first model pipeline let's divide data into training and test sets

In [4]:
from sklearn.cross_validation import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=1)

### Combining transformers and estimators in a pipeline

As we learned previously, we want to standardize the features of the model before running it. So, let's assume that we want to compress our data from 30 dimensions onto a lower two-dimensional subspace via PCA. Instead of going through the transformation steps we can chain the **StandardScaler**, **PCA** and **LogisticRegression** in a pipeline

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
pipe_lr = Pipeline([('scl', StandardScaler())])